In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import pickle as pkl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression 
from sklearn.model_selection import train_test_split
import joblib

%run ../dnam-clocks/mFSS-clock/scripts/mfss_model_train.py
%matplotlib inline

In [4]:
data_path = '../dnam-clocks/mFSS-clock/data/processed/'

In [ ]:
# Read in the Hannum dataset for training
hannum = pd.read_pickle(data_path + 'GSE40279_hannum/hannum32.pkl')
hannum_meta = pd.read_excel('GSE40279_hannum/hannum_pmeta.xlsx')

In [5]:
# Read in the Lehne datasat for validation
#lehne = pd.read_pickle(data_path + 'GSE55763_lehne/lehne_reduced.pkl')
lehne_meta = pd.read_excel(data_path + 'GSE55763_lehne/lehne_pmeta.xlsx')

In [7]:
#get the age_correlations so we can focus on the shifts of just the top N age-correlated CpGs
age_corrs=pd.read_excel(data_path+ 'cpg_sets/hannum_age_corrs.xlsx')

In [9]:
#filter out SNPS
manifest = pd.read_csv(data_path + 'metadata/450K_manifest.csv', low_memory=False)
# Get all SNP-associated CpGs in 450K array 
snps = set(manifest[~manifest.Probe_SNPs.isna()].Name)
no_snps = list(set(age_corrs.CpG)-snps)
age_corrs = age_corrs[age_corrs['CpG'].isin(no_snps)]

In [ ]:
#Lehne has some missing CpGs so need to remove those from the age_corrs rows
overlap = list(set(lehne.columns)&set(age_corrs.CpG))

filtered_age_corrs= age_corrs[age_corrs['CpG'].isin(overlap)]
filtered_age_corrs.reset_index(inplace=True)

In [6]:
lehne_meta

,Unnamed: 0,key,ID_REF,disease_state,gender,age,series_id
0,0,7471147002_R01C01,GSM1343122,Population study sample.,M,37.0,GSE55763
1,1,7471147002_R01C02,GSM1343123,Population study sample.,M,43.8,GSE55763
2,2,7471147002_R02C01,GSM1343124,Population study sample.,M,47.4,GSE55763
3,3,7471147002_R02C02,GSM1343125,Population study sample.,F,38.6,GSE55763
4,4,7471147002_R03C01,GSM1343126,Population study sample.,M,44.3,GSE55763
...,...,...,...,...,...,...,...
2626,2634,7796814021_R04C02,GSM1345756,Population study sample.,M,52.7,GSE55763
2627,2635,7796814021_R05C01,GSM1345757,Population study sample.,M,44.3,GSE55763
2628,2636,7796814021_R05C02,GSM1345758,Population study sample.,F,46.4,GSE55763
2629,2637,7796814021_R06C01,GSM1345759,Population study sample.,M,39.5,GSE55763


In [ ]:
model_cgs, best_iter, val_mse, val_r_val, test_mse, test_r_val = mf_stepwise(filtered_age_corrs.CpG.tolist()[:10000], hannum, hannum_meta,
                                                                             lehne, lehne_meta, 150,flag=False)